In [569]:
"""
importing useful libraries
pandas: manipulating tables and arrays
numpy: manipulate arrays (usually for faster computation than structures like lists)
keyring & teradatasql: saving user credentials and connecting to teradata
pandasql: run sql queries with pandas dataframes
time: calculate run time
datetime: converting strings to datetime objects and vice versa
os: interact with the directories (folder) on this computer
jupyternotify: notifies when cell execution has occurred
"""

import pandas as pd
#import modin.pandas as pd
from pandas import read_sql, ExcelWriter, DataFrame, Series, to_datetime, concat
import numpy as np
from numpy import zeros, where, mean, vectorize, round, ceil
import keyring
import teradatasql
import pandasql
from pandasql import sqldf
import time
import datetime
import os
import saspy
import math
import warnings
import getpass

In [571]:
promo_ad_year = 2024 #integer
promo_ad_week = 52 #integer
current_week = 42
quarter_num = 1

In [573]:
username = 'carshen'

In [575]:
# PASSWORD: 
try:
    password = getpass.getpass()
except:
    print("fail: ", error)
else:
    print("TERADATA PASSWORD")

 ········


TERADATA PASSWORD


In [577]:
#to set up username and password so you dont need to type it
#only needs to be run once ever


host = 'TDPROD1.ngco.com'

# password=keyring.get_password("TD", username)\

queries_folder = os.path.abspath(os.getcwd()) + '\\SQL Queries'

# output_folder = os.path.abspath(os.getcwd()) + '\\Output'

con = teradatasql.connect(None, host=host, user=username, password=password, logmech = 'LDAP')

In [701]:
df = pd.DataFrame({'Article_ID': [], 'Fixture_ID':[], 'Fixture_position':[]})
def get_article_data():
    global df
    article_ids = input("Enter Article IDs separated by spaces (e.g., '12345 67890 98765'): ").split()

    for article_id in article_ids:
        fixture_id = input(f"Enter Fixture ID for Article {article_id}: ")
        fixture_position = input(f"Enter Fixture Position for Article {article_id}: ")

        new_row = {'Article_ID': article_id,
                   'Fixture_ID': fixture_id,
                   'Fixture_position': fixture_position}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    print(df.to_string(index=False))

get_article_data()
#NOTE: if items are going into different fixtures for different stores please do seperate inputs

Enter Article IDs separated by spaces (e.g., '12345 67890 98765'):  21124364
Enter Fixture ID for Article 21124364:  104
Enter Fixture Position for Article 21124364:  1


Article_ID Fixture_ID Fixture_position
  21124364        104                1


In [703]:
article_ids_str = ", ".join("'" + str(id) + "'" for id in df['Article_ID'])
hier_query = f'''
SELECT DISTINCT
TRIM(ARTCL_NUM) AS ARTCL_NUM, 
ARTCL_MED_EN_DESC, 
VOLUME_QUANTITY, 
VOLUME_UNIT, 
CASE_PACK_QUANTITY 
FROM RLDMPROD_V.ARTCL_MCH_HIER_ALL  
WHERE ARTCL_NUM IN ({article_ids_str}) 
'''
hier_query_db = pd.read_sql(hier_query, con)
#hier_query_db

C:\Users\carshen\AppData\Local\Temp\ipykernel_12100\2576150109.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hier_query_db = pd.read_sql(hier_query, con)


In [705]:
hier_query_db['ARTCL_NUM'] = pd.to_numeric(hier_query_db['ARTCL_NUM'], errors='coerce')

df['Article_ID'] = pd.to_numeric(df['Article_ID'], errors='coerce')
hier_query_db['ARTCL_NUM'] = pd.to_numeric(hier_query_db['ARTCL_NUM'], errors='coerce')

merged_df = pd.merge(df, hier_query_db, 
                     left_on='Article_ID', 
                     right_on='ARTCL_NUM', 
                     how='left')
#merged_df

In [707]:
def fixture_size_format(fixture_id):
    #GROCERY PP
    if fixture_id == 67: 
        return (6*2.54,24*2.54,56*2.54)
    #NATURAL FOODS PP
    elif fixture_id == 68: 
        return (6*2.54,14*2.54,52*2.54)
    #18X18 BINS
    elif fixture_id in (61,62,63,64,65,66):
        return (18.5*2.54,18.5*2.54,15.5*2.54)
    #TABLE
    elif fixture_id in (5,11): 
        return (36*2.54,36*2.54,18*2.54) 
    #FRONT ENDCAP, BACK ENDCAP, MCM ENDCAP
    elif fixture_id in (1,3,4): 
        return (48*2.54,18*2.54,72*2.54)
    #ONE ENDCAP SHELF
    elif fixture_id == 100: 
        return (48*2.54,18*2.54,45.72) 
    #TWO ENDCAP SHELVES
    elif fixture_id == 101:
        return (48*2.54,18*2.54,91.44)
    #LANDINI RACKS
    elif fixture_id in (71,72,73,74,75,76): 
        return (48*2.54,12*2.54,36*2.54)
    #GROCERY/NV | PALLETS | PALLET
    elif fixture_id in (6,12): 
        return (48*2.54,40*2.54,60*2.54) #48,40,60
    #BUNKER SIDE
    elif fixture_id == 99: 
        return (36*2.54, 30*2.54, 12*2.54)
    #HALF OF BUNKER SIDE
    elif fixture_id == 104:
        return (18*2.54, 30*2.54, 12*2.54)
    #BUNKER END
    elif fixture_id == 102:
        return (72*2.54, 30*2.54, 12*2.54)
    #UPRIGHT/DOOR
    elif fixture_id == 98: 
        return (24*2.54, 18*2.54, 72*2.54)
    #9FT BUNKER SIDE
    elif fixture_id == 103:
        return (9*12*2.43, 30*2.54, 12*2.54)
    else:
        print("error")
        return None
        
def apply_fixture_size(row):
    sizes = fixture_size_format(row['Fixture_ID'])
    if sizes is not None:
        return sizes
    else:
        return (None,None,None,None)
        
def fixture_name_format(fixture_id):
    fixture_id = int(fixture_id)
    if fixture_id == 67:
         return "GROCERY | ENDCAP | PROFIT PANEL"
    elif fixture_id == 68:
        return "NATURAL FOODS | ENDCAP | PROFIT PANEL"
    elif fixture_id == 61:
        return "DAIRY | BUNKER | 18 X 18 BIN"
    elif fixture_id == 62:
        return "FROZEN | BUNKER | 18 X 18 BIN"
    elif fixture_id == 63:
        return "SPLIT DAIRY FROZEN | BUNKER | 18 X 18 BIN"
    elif fixture_id == 64:
        return "MEAT | BUNKER | 18 X 18 BIN"
    elif fixture_id == 65:
        return "SEAFOOD | BUNKER | 18 X 18 BIN"
    elif fixture_id == 66:
        return "GROCERY | NON BUNKER | 18 X 18 BIN"
    elif fixture_id == 5:
        return "GROCERY | TABLES | TABLE"
    elif fixture_id == 11:
        return "NV | TABLES | TABLE"
    elif fixture_id == 1:
        return "GROCERY | FRONT ENDCAP | SHELF"
    elif fixture_id == 3:
        return "GROCERY | BACK ENDCAP | SHELF"
    elif fixture_id == 4:
        return "GROCERY | MULTICULTURAL ENDCAP | SHELF"
    elif fixture_id == 100:
        return "ENDCAP | SHELF"
    elif fixture_id == 6:
        return "GROCERY | PALLETS | PALLET"
    elif fixture_id == 12:
        return "NV | PALLETS | PALLET"
    elif fixture_id == 99:
        return "FROZEN | BUNKER | SIDE"
    elif fixture_id == 98:
        return "FROZEN | UPRIGHT | DOOR"
    elif fixture_id == 101:
        return "ENDCAP | TWO SHELVES"
    elif fixture_id == 102:
        return "BUNKER | END"
    elif fixture_id == 103:
        return "BUNKER | SIDE | 9FT"
    elif fixture_id == 104:
        return "HALF OF BUNKER SIDE"
    #LANDINI RACKS/PALLETS NOT HERE
    else:
        print("error")
        return None

def fixture_name_populate():
    fixtures = merged_df.loc[:,'Fixture_ID'].tolist()
    fixture_name_list = []

    for fixture_id in fixtures:
        fixture_name = fixture_name_format(fixture_id)
        fixture_name_list.append(fixture_name)
    return fixture_name_list

def fixture_vol_calculate(row):
    length = row.get('Length(CM)')
    width = row.get('Width(CM)')
    height = row.get('Height(CM)')

    if all((pd.notna(length), pd.notna(width), pd.notna(height))):
        volume = length * width * height
        return round(volume, 2)
    else:
        return 0 

'''def num_of_items_calculate(data):
    result = data.groupby('Fixture_ID')['Fixture_position'].size().reset_index(name = 'num of items')
    data = data.merge(result, on = 'Fixture_ID')
    return data
'''

def num_of_items_calculate(data):
    counts = []
    item_counts = data.groupby(['Fixture_ID', 'Fixture_position'])['Article_ID'].transform('size')
    min_qty_df['Number_of_Items'] = item_counts
    return min_qty_df['Number_of_Items']
       
def min_qty_calculate(row):
    fixture_vol = row.get('Fixture_Volume (CCM)')
    item_vol = row.get('Item_Volume(CCM)')
    num_of_items = row.get('Number_of_Items')
    min_qty = fixture_vol/item_vol/num_of_items
    return round(min_qty,0)

def min_case_pack_qty_calculate(row):
    min_qty = row.get('Min_Qty')
    case_pack_qty = row.get('Case_Pack_Qty')
    min_case_pack_qty = min_qty/case_pack_qty
    return round(min_case_pack_qty,0)
    
    

In [709]:
min_qty_df = pd.DataFrame()

In [711]:
min_qty_df['Article_ID'] = merged_df['Article_ID']
min_qty_df['Article_Desc'] = merged_df['ARTCL_MED_EN_DESC'].str.strip()
min_qty_df['Fixture_ID'] = merged_df['Fixture_ID'].str.strip().astype('int64')
min_qty_df['Fixture_position'] = merged_df['Fixture_position'].str.strip().astype('int64')
min_qty_df['Fixture_Name'] = fixture_name_populate()
min_qty_df['Case_Pack_Qty'] = merged_df['CASE_PACK_QUANTITY']
min_qty_df[['Length(CM)','Width(CM)','Height(CM)']] = min_qty_df.apply(apply_fixture_size, axis=1, result_type = "expand")
min_qty_df['Fixture_Volume (CCM)'] = min_qty_df.apply(fixture_vol_calculate, axis = 1)
num_of_items_calculate(min_qty_df) #Number_of_Items column
min_qty_df['Item_Volume(CCM)'] = merged_df['VOLUME_QUANTITY']
min_qty_df['Min_Qty'] = min_qty_calculate(min_qty_df)
min_qty_df['Min_Case_Pack_Qty'] = min_case_pack_qty_calculate(min_qty_df)
min_qty_df

,Article_ID,Article_Desc,Fixture_ID,Fixture_position,Fixture_Name,Case_Pack_Qty,Length(CM),Width(CM),Height(CM),Fixture_Volume (CCM),Number_of_Items,Item_Volume(CCM),Min_Qty,Min_Case_Pack_Qty
0,21124364,PC SPICY HUMMUS,104,1,HALF OF BUNKER SIDE,12.0,45.72,76.2,30.48,106188.17,1,595.125,178.0,15.0


min_qty_df

In [565]:
min_qty_file_name = input("File name: ") #uncomment when need seperate min qty file

File name:  may hotm min qty check NEW march 3


In [567]:

with ExcelWriter(str(min_qty_file_name) + '.xlsx') as writer:
    min_qty_df.to_excel(writer)


In [555]:
# find AH_4_CD for each Article_ID in min_qty_df 
article_ids_str = ", ".join("'" + str(id) + "'" for id in min_qty_df['Article_ID'])
AH_4_CD_query = f'''
SELECT
T1.ARTCL_NUM,
T1.AH_4_CD,
T1.AH_4_MED_DESC_EN,
T2.ARTCL_NUM,
T2.VOLUME_QUANTITY,
T2.VOLUME_UNIT,
T2.BSE_UT_SZ_QTY,
T2.BSE_UT_SZ_UOM_CD
FROM RLDMPROD_V.ARTCL_AH_HIER T1
LEFT JOIN RLDMPROD_V.ARTCL_MCH_HIER_ALL T2 
ON T1.ARTCL_NUM = T2.ARTCL_NUM
WHERE T1.ARTCL_NUM IN ({article_ids_str})
'''
AH_4_DC_DF = pd.read_sql(AH_4_CD_query, con)
#AH_4_DC_DF #comment out

C:\Users\carshen\AppData\Local\Temp\ipykernel_12100\3336511027.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AH_4_DC_DF = pd.read_sql(AH_4_CD_query, con)


In [557]:
composite_key_list = []
for index, row in AH_4_DC_DF.iterrows():
    key = (row['AH_4_CD'], row['BSE_UT_SZ_QTY'], row['BSE_UT_SZ_UOM_CD'].strip())
    composite_key_list.append(key)
    
composite_key_string = ", ".join([f"('{k1}', '{k2}', '{k3}')" for k1, k2, k3 in composite_key_list])

where_clauses = []
for k1, k2, k3 in composite_key_list:
    # Calculate the 15% tolerance range for k2
    lower_bound = k2 * 0.85  # 85% of k2
    upper_bound = k2 * 1.15  # 115% of k2
    
    # Add the condition for T2.BSE_UT_SZ_QTY being within 15% of k2
    where_clauses.append(f"(T1.AH_4_CD = '{k1}' AND "
                         f"T2.BSE_UT_SZ_QTY BETWEEN {lower_bound} AND {upper_bound} "
                         f"AND T2.BSE_UT_SZ_UOM_CD = '{k3}')")

where_clause = " OR ".join(where_clauses)
    
volume_check_query = f"""
    SELECT DISTINCT
    T1.ARTCL_NUM,
    T1.AH_4_CD,
    T1.AH_4_MED_DESC_EN,
    T2.ARTCL_NUM,
    T2.ARTCL_MED_EN_DESC,
    T2.VOLUME_QUANTITY,
    T2.VOLUME_UNIT,
    T2.BSE_UT_SZ_QTY,
    T2.BSE_UT_SZ_UOM_CD
    FROM RLDMPROD_V.ARTCL_AH_HIER T1
    LEFT JOIN RLDMPROD_V.ARTCL_MCH_HIER_ALL T2 ON T1.ARTCL_NUM = T2.ARTCL_NUM
    WHERE {where_clause}
    """

# Execute the query and retrieve the results into a DataFrame
AH_4_merged_df = pd.read_sql(volume_check_query, con)
#AH_4_merged_df

C:\Users\carshen\AppData\Local\Temp\ipykernel_12100\3214413926.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AH_4_merged_df = pd.read_sql(volume_check_query, con)


In [559]:
AH_4_group_counts = AH_4_merged_df.groupby(['AH_4_CD', 'BSE_UT_SZ_QTY', 'BSE_UT_SZ_UOM_CD']).size().reset_index(name='count')
#AH_4_group_counts

In [561]:
def calculate_avg_volume_with_outlier_removal(df: pd.DataFrame) -> pd.DataFrame:

    def calculate_avg_without_outliers(group):
        """Calculates average after removing outliers using the IQR method."""
        q1 = group['VOLUME_QUANTITY'].quantile(0.25)
        q3 = group['VOLUME_QUANTITY'].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        filtered_group = group[(group['VOLUME_QUANTITY'] >= lower_bound) & (group['VOLUME_QUANTITY'] <= upper_bound)]
        return pd.Series({
            'Avg_VOLUME_QUANTITY': filtered_group['VOLUME_QUANTITY'].mean(),
            'count': len(filtered_group),  # Count rows after outlier removal
            'StDev_VOLUME_QUANTITY': filtered_group['VOLUME_QUANTITY'].std()
        }) 

     # Calculate average volume, count, and standard deviation (as percentage)
    avg_volume_df = df.groupby(
        ['AH_4_CD', 'BSE_UT_SZ_QTY', 'BSE_UT_SZ_UOM_CD']
    ).apply(calculate_avg_without_outliers).reset_index()

    # Calculate standard deviation as a percentage of the average
    avg_volume_df['StDev_VOLUME_QUANTITY_Percent'] = (
        avg_volume_df['StDev_VOLUME_QUANTITY'] / avg_volume_df['Avg_VOLUME_QUANTITY'] * 100
    )
    avg_volume_df.drop('StDev_VOLUME_QUANTITY', axis=1, inplace=True)

    return avg_volume_df

avg_volume_by_ah4cd = calculate_avg_volume_with_outlier_removal(AH_4_merged_df)
#avg_volume_by_ah4cd

C:\Users\carshen\AppData\Local\Temp\ipykernel_12100\2825791338.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(calculate_avg_without_outliers).reset_index()


In [563]:
def compare_volumes(AH_4_DC_DF: pd.DataFrame, avg_volume_by_ah4cd: pd.DataFrame)-> pd.DataFrame:
    # Merge the two DataFrames for comparison
    merged_df = pd.merge(
        AH_4_DC_DF,
        avg_volume_by_ah4cd,
        on=['AH_4_CD', 'BSE_UT_SZ_QTY', 'BSE_UT_SZ_UOM_CD'],
        how='left'
    )

    def calculate_quantile(group):
        """Calculates quantiles within each group."""
        q1 = group['Avg_VOLUME_QUANTITY'].values[0] * 0.5 
        q3 = group['Avg_VOLUME_QUANTITY'].values[0] * 1.5  

        # Define quantile labels
        quantile_labels = ['Q1 • <=0.5', 'Q2', 'Q3 • >=1.5']

        group['Notification'] = pd.cut(group['VOLUME_QUANTITY'], 
                                              bins=[float('-inf'), q1, q3, float('inf')],
                                              labels=quantile_labels,
                                              right=False) 
         # Add "low count avg" notification 
        if group['count'].values[0] < 10:
            group['count'] = group['count'].astype(str) + ' • low count avg' 
        return group

    merged_df = merged_df.groupby(['AH_4_CD', 'BSE_UT_SZ_QTY', 'BSE_UT_SZ_UOM_CD']).apply(calculate_quantile)
    merged_df = merged_df.set_index(AH_4_DC_DF.index)
    AH_4_DC_DF['Notification'] = merged_df['Notification']

    return merged_df

result_df = compare_volumes(AH_4_DC_DF.copy(), avg_volume_by_ah4cd.copy())
result_df

C:\Users\carshen\AppData\Local\Temp\ipykernel_12100\3479827937.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby(['AH_4_CD', 'BSE_UT_SZ_QTY', 'BSE_UT_SZ_UOM_CD']).apply(calculate_quantile)


,ARTCL_NUM,AH_4_CD,AH_4_MED_DESC_EN,ARTCL_NUM,VOLUME_QUANTITY,VOLUME_UNIT,BSE_UT_SZ_QTY,BSE_UT_SZ_UOM_CD,Avg_VOLUME_QUANTITY,count,StDev_VOLUME_QUANTITY_Percent,Notification
0,21092033,233945,Deli Ambient ...,21092033,2365.951,CCM,200.0,G,1298.915270,37.0,82.856390,Q3 • >=1.5
1,20500992,233946,Deli Dips ...,20500992,660.643,CCM,250.0,G,693.218786,42.0,8.502302,Q2
2,20500991,233946,Deli Dips ...,20500991,685.234,CCM,250.0,G,693.218786,42.0,8.502302,Q2
3,20501051,233946,Deli Dips ...,20501051,660.643,CCM,255.0,G,463.079900,10.0,75.169943,Q2
4,20594792,233946,Deli Dips ...,20594792,660.643,CCM,255.0,G,463.079900,10.0,75.169943,Q2
5,20501133,233946,Deli Dips ...,20501133,678.935,CCM,260.0,G,671.726933,30.0,1.376339,Q2
6,20501014,233946,Deli Dips ...,20501014,678.935,CCM,260.0,G,671.726933,30.0,1.376339,Q2
7,20501132,233946,Deli Dips ...,20501132,685.234,CCM,260.0,G,671.726933,30.0,1.376339,Q2
8,20562771,233946,Deli Dips ...,20562771,678.935,CCM,260.0,G,671.726933,30.0,1.376339,Q2
9,20811328,233946,Deli Dips ...,20811328,883.050,CCM,280.0,G,668.868333,12.0,8.142568,Q2


In [37]:
volume_check = input("file name: ")

file name:  EASTER MIN QTY VOLUME CHECK


In [39]:
# uncomment for min-qty file
with ExcelWriter(str(volume_check) + '.xlsx') as writer:
    result_df.to_excel(writer)

In [713]:
'''
Fixture check for ALL sites need to be done prior to input
Check for missing fixtures, closed stores, stores rename, etc
'''
Stores = input("Store #: ").split()

Store #:  309 311 312 313 314 315 317 318 325 326 329 331 332 336 337 339 341 344 350 351 352 353 354 358 359 364 366 367 368 369 372 373 374 375 376 377 378 380 384 385 386 388 389 396 405 507 509 515 521 525 528 529 533 536 537 538 539 540 543 550 552 557 559 560 563 565 573 576 579 580 806 810 827 835 839 866 878 895 906 922 934 1000 1001 1003 1004 1007 1010 1011 1014 1016 1019 1021 1022 1023 1027 1028 1029 1032 1035 1040 1050 1051 1066 1079 1082 1083 1090 1099 1114 1127 1132 1142 1170 1188 1194 1212 1424 1627 1659 1664 1734 1789 1799 1806 1948 2671 2683 2719 4430 6799 6951 7115 7155 7161 7164 7179 7195 7198 7288 7293 7295 7297 7309 7316 7494 7506 7534 7599 7606 7907 7908 7910 7914 8180 8807 8978


In [593]:
pip install pyxlsb

In [715]:
stores_df = pd.DataFrame({'Store #': [], 'Store Desc': []})
def location_reader():
    global stores_df
    store_list = pd.read_excel("Stores List.xlsb", engine='pyxlsb')
    stores_df = store_list
    return stores_df

def banner_desc(final_df, stores_df):
    final_df['Store #'] = final_df['Store #'].astype(int)
    stores_df['Store #'] = stores_df['Store #'].astype(int)
    store_banner_mapping = dict(zip(stores_df['Store #'], stores_df['Store Desc']))
    final_df['Banner'] = final_df['Store #'].map(store_banner_mapping)
    return final_df

def articles_to_stores(min_qty_df, mpq_df):
    new_data = []
    for _, store_row in mpq_df.iterrows():
        store_num = store_row['Store #']
        #banner = store_row['Banner']
        eff_date = store_row['*EFF date']
        disc_date = store_row['*DISC date']
        requested_by = store_row['Requested by']

        for _, article_row in min_qty_df.iterrows():
            new_row = {
                'Location': article_row['Fixture_Name'],
                'Article': article_row['Article_ID'],
                'Desc': article_row['Article_Desc'],
                'Store #': store_num,
                '*EFF date': eff_date,
                '*DISC date': disc_date,
                'Display Qty(units)': article_row['Min_Qty'],  
                'Comments': '',  # Or you could add logic for comments
                'Requested by': requested_by,
                #'Banner': banner
            }
            new_data.append(new_row)
    global df_combined
    df_combined = pd.DataFrame(new_data)
    return df_combined     

In [717]:
location_reader()

,Store #,Store Desc
0,309,Atlantic Superstore
1,311,Atlantic Superstore
2,312,Atlantic Superstore
3,313,Atlantic Superstore
4,314,Atlantic Superstore
...,...,...
356,1423,Atl Your Ind Grocer
357,1887,Your Ind Grocer
358,1417,Atl Your Ind Grocer
359,7926,Your Ind Grocer


In [719]:
mpq_df = pd.DataFrame()

In [335]:
Requested_by = input("Requested by: ")
eff_date = input("*EFF date: ")
disc_date = input("*DISC date: ")

Requested by:  NICOLAS CAMACHO
*EFF date:  2025/5/1
*DISC date:  2025/5/31


In [721]:
mpq_df['Location'] = 0
mpq_df['Article'] = 0
mpq_df['Desc'] = 0
mpq_df['Store #'] = Stores
mpq_df['*EFF date'] = eff_date
mpq_df['*DISC date'] = disc_date
mpq_df['Display Qty(units) '] =0 
mpq_df['Comments'] = " "
mpq_df['Requested by'] = Requested_by
#banner_desc(mpq_df, stores_df) #Banner column

In [723]:
final_df = articles_to_stores(min_qty_df, mpq_df)
final_df
#banner_desc(df_combined, stores_df)

,Location,Article,Desc,Store #,*EFF date,*DISC date,Display Qty(units),Comments,Requested by
0,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,309,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO
1,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,311,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO
2,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,312,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO
3,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,313,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO
4,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,314,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO
...,...,...,...,...,...,...,...,...,...
151,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,7910,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO
152,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,7914,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO
153,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,8180,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO
154,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,8807,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO


In [725]:
banner_desc(final_df, stores_df)

,Location,Article,Desc,Store #,*EFF date,*DISC date,Display Qty(units),Comments,Requested by,Banner
0,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,309,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO,Atlantic Superstore
1,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,311,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO,Atlantic Superstore
2,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,312,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO,Atlantic Superstore
3,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,313,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO,Atlantic Superstore
4,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,314,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO,Atlantic Superstore
...,...,...,...,...,...,...,...,...,...,...
151,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,7910,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO,Provigo
152,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,7914,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO,Your Ind Grocer
153,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,8180,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO,Provigo
154,HALF OF BUNKER SIDE,21124364,PC SPICY HUMMUS,8807,2025/5/1,2025/5/31,178.0,,NICOLAS CAMACHO,Provigo


In [727]:
file_name = input("File name: ")

File name:  MARCH 5TH PC SPICY HUMMUS ADQ


In [729]:
with ExcelWriter(str(file_name) + '.xlsx') as writer:
    df_combined.to_excel(writer)